# Python-Notebook für SEPA-Lastschrift-Einzugsnachrichten

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

## Setup

Um dieses Notebook auszuführen, müssen alle Dependencies in der `requirements.txt` installiert werden. Dies kann mit folgendem Befehl gemacht werden:

```bash
pip install -r requirements.txt
```

## Import der Mitglieder, Beiträge und Lastschriftmandate

In diesem Abschnitt werden die Mitgliederdaten und Zahlungsinformationen aus der Mitgliederliste importiert.

### Vorbereitung

Zur Vorbereitung müssen folgende Schritte gemacht werden:
1. Erstelle die Ordner `data` im Projektordner, falls er nicht schon vorhanden ist.
2. Kopiere die Mitgliederliste aus der JuBO-Cloud unter `Vorstand > Mitglieder > JuBO-Mitglieder.xlsx` in den lokalen Ordner `data`.

Die Ordnerstruktur sollte dann wie folgt aussehen:
```
<project_root>
├── data/
|   └── JuBO-Mitglieder.xlsx
└── [...]
```

**Wichtig**: Die Datei darf nicht umbenannt werden, da sie sonst nicht gefunden wird.

### Routine

Die Daten aus der Tabelle werden wie folgt verarbeitet:

1. Die Mitgliedsdaten im Sheet "Mitglieder" und Zahlungsdaten im Sheet "Zahlungsdaten" werden aus der Tabelle ausgelesen.
2. Da nur aktive Mitglieder den Mitgliedsbeitrag zahlen, werden alle Mitglieder aus den Daten herausgefiltert, die nicht den Status "Aktiv" haben.
3. Anschließend wird überprüft, ob für jedes zahlende Mitglied ein SEPA-Lastschriftmandat vorliegt, indem geprüft wird, ob die Spalte "Erteilt Am" unter "Lastschriftmandat" ausgefüllt ist. Falls nicht, wird das Mitglied aus der Liste entfernt und eine Warnung ausgegeben.
4. Die Mitglieds- und Zahlungsdaten werden zusammengefügt.


In [ ]:
import pathlib
from src.paths import DATA_DIR

# Set the path to the member data file
MEMBER_FILE_PATH = pathlib.Path(DATA_DIR, 'JuBO-Mitglieder.xlsx')
assert MEMBER_FILE_PATH.exists(), f"File not found: \"{MEMBER_FILE_PATH}\"."
print("Member File Path:", MEMBER_FILE_PATH.resolve())

In [ ]:
from src.excel import read_excel

# Load the members form the "Mitglieder" worksheet
member_data = read_excel(
    workbook_path=MEMBER_FILE_PATH,
    sheet_name="Mitglieder",
    header_map={
        'ID': 'id',
        'Anrede': 'salutation',
        'Vorname': 'first_name',
        'Nachname': 'last_name',
        'E-Mail': 'email',
        'Status': 'status',
        'Mitgliedschaft': 'member_type'
    },
    skip_rows=4
)

# Load the payment information from the "Finanzen" worksheet
payment_data = read_excel(
    workbook_path=MEMBER_FILE_PATH,
    sheet_name="Finanzen",
    header_map={
        'MitgliedsNr.': 'member_id',
        'Beitrag': 'amount_fee',
        'Spende': 'amount_donation',
        'Gesamt': 'amount_total',
        'Voller Name': 'account_holder',
        'Referenz': 'mandate_reference',
        'Gläubiger-ID': 'creditor_id',
        'Erteilt Am': 'issue_date',
        'IBAN (Anonymisiert)': 'iban_anonymized',
        'BIC (Anonymisiert)': 'bic_anonymized',
        'Kreditinstitut': 'credit_institute'
    },
    skip_rows=4
)
# Convert NaN to zero in amount fields and convert the mandate reference to a string
payment_data['amount_donation'] = payment_data['amount_donation'].fillna(0)
payment_data['amount_fee'] = payment_data['amount_fee'].fillna(0)
payment_data['amount_total'] = payment_data['amount_total'].fillna(0)
payment_data['mandate_reference'] = payment_data['mandate_reference'].fillna(0)
payment_data['mandate_reference'] = payment_data['mandate_reference'].astype(int).astype(str)

# Filter members that are still part of the club, i.e. have the status "Aktiv"
active_members = member_data[member_data["status"].isin(["Aktiv"])]

# Merge the active members with the payment data
payments = active_members.merge(
    payment_data,
    how='left',
    left_on='id',
    right_on='member_id'
)
payments

In [ ]:
# Find members that didn't issue a SEPA mandate
payments_with_mandate = payments[payments['issue_date'].notna()]
payments_without_mandate = payments[payments['issue_date'].isna()]
if len(payments_without_mandate) > 0:
    print(f"Found {len(payments_without_mandate)} members without a SEPA mandate. Contect them separately.")
payments_without_mandate

In [ ]:
# Display the total amounts
print(f'Number of paying members: {len(payments_with_mandate)}/{len(payments_with_mandate) + len(payments_without_mandate)}')
print(f'Fees:\t\t{payments_with_mandate["amount_fee"].sum()}€')
print(f'Donations:\t{payments_with_mandate["amount_donation"].sum()}€')
print(f'Total:\t\t{payments_with_mandate["amount_total"].sum()}€')

## Erstellen der E-Mails

In diesem Abschnitt werden die E-Mails für die Ankündigung des Lastschrifteinzugs aus den importierten Daten erstellt.

### Vorbereitung

1. Kopiere die Datei `.env.example`, die sich im Projektordner befindet, und benenne sie in `.env` um.
2. Konfiguriere die Parameter in der Datei `.env` wie folgt:

| Name | Beschreibung | Standartwert | Erforderlich? |
| --- | --- | --- | --- |
| EMAIL_USER | Die E-Mail-Adresse für den Login, z.B. *schatzmeister@jubo.info*. Gleich mit dem Login oder in Outlook. |  | Ja |
| EMAIL_PASSWORD | Das Passwort für den Login, z.B. *Passwort123*. | | Ja |
| EMAIL_IMAP_HOST | Die Domain des IMAP E-Mail-Servers. | imap.app.magix-online.com | Ja |
| EMAIL_IMAP_PORT | Der Port des IMAP E-Mail-Servers. | 993 | Ja |
| EMAIL_SMTP_HOST | Der Domain des SMTP E-Mail-Servers. | smtp.app.magix-online.com | Ja |
| EMAIL_SMTP_PORT | Der Port des SMTP E-Mail-Servers. | 465 | Ja |
| SIGNATURE_NAME | Der Name des Absenders, z.B. "Berry Gauer". Wird in die Signatur gesetzt. | | Ja |
| SIGNATURE_ROLE | Die Rolle des Absenders im Vorstand, z.B. "1. Vorstandsvorsitzender der JuBO e.V.". Wird in der Signatur unter dem Namen hinzugefügt. | | Ja |
| SIGNATURE_EMAIL | Die Kontakt-E-Mail des Absenders, z.B. "vorstand@jubo.info" | | Ja |
| SIGNATURE_PHONE | Die Telefonnummer des Abesenders, z.B. "0123/4567890" | | Nein |

Die Ordnerstruktur sollte dann wie folgt aussehen:
```
<project_root>
├── data/
|   └── JuBO-Mitglieder.xlsx
├── .env
├── .env.example
└── [...]
```

### Routine

Die E-Mails werden wie folgt erstellt:

1. Die vorbereitete Konfiguration wird aus der Datei `.env` geladen.
2. Die E-Mail HTML-Vorlage wird aus der Datei `templates/mitgliedsbeiträge/mitgliedsbeiträge.html.jinja` geladen.
3. Für jedes wird mit der Vorlage eine E-Mail erstellt, indem die jeweiligen Daten des Mitglieds in die Vorlage eingefügt werden.

In [ ]:
import pathlib
from src.paths import ROOT_DIR

#  Set the path to the dotenv file
DOTENV_PATH = pathlib.Path(ROOT_DIR, '.env')
assert DOTENV_PATH.exists(), f"File not found: \"{DOTENV_PATH}\"."
print("Dotenv File Path:", DOTENV_PATH.resolve())

In [ ]:
from src.email import EmailClientConfig, EmailSignatureConfig

# Load the email client and signature configuration from the .env file
client_config = EmailClientConfig.from_dotenv(DOTENV_PATH)
signature_config = EmailSignatureConfig.from_dotenv(DOTENV_PATH)

In [ ]:
from src.jinja import read_template

# Set the name of the template file, which is the path relative to the templates directory
TEMPLATE_NAME = 'mitgliedsbeiträge/mitgliedsbeiträge.html.jinja'

# Load the template
template = read_template(TEMPLATE_NAME, globals={
    "format_date": lambda x: x.strftime("%d.%m.%Y"),
    "format_currency": lambda x: "{:,.2f}€".format(x).replace(".", ",")
})

In [ ]:
from datetime import datetime, timedelta

# Get the current year
CURRENT_YEAR = datetime.now().year

# Set the update deadline to 7 days from now
UPDATE_DATE = datetime.now() + timedelta(days=7)
print()

# Set the collection date to 14 days from now
COLLECTION_DATE = datetime.now() + timedelta(days=14)
print(
    "Current Year:", CURRENT_YEAR,
    " | Update Deadline:", UPDATE_DATE.strftime('%d.%m.%Y'),
    " | Collection Date:", COLLECTION_DATE.strftime('%d.%m.%Y')
)

In [ ]:
from IPython.display import display, HTML
from src.email import Email

# Prepare the e-mails
emails: list[Email] = []
for payment in payments_with_mandate.itertuples():
    content = template.render(
        salutation=payment.salutation,
        first_name=payment.first_name,
        member_type=payment.member_type,
        amount_fee=payment.amount_fee,
        amount_donation=payment.amount_donation,
        amount_total=payment.amount_total,
        account_holder=payment.account_holder,
        iban_anonymized=payment.iban_anonymized,
        bic_anonymized=payment.bic_anonymized,
        mandate_reference=payment.mandate_reference,
        creditor_id=payment.creditor_id,
        issue_date=payment.issue_date,
        year=CURRENT_YEAR,
        update_date=UPDATE_DATE,
        collection_date=COLLECTION_DATE,
        contact_email=client_config.user,
        signature_name=signature_config.name,
        signature_role=signature_config.role,
        signature_email=signature_config.email,
        signature_phone=signature_config.phone
    )
    email = Email(
        sender=client_config.user,
        to=[payment.email],
            subject=f'JuBO e.V. | Mitgliedsbeitrag { CURRENT_YEAR } | Mitglied Nr. M{ payment.id } { payment.first_name } { payment.last_name }',
        content=content
    )
    emails.append(email)

# Display the first e-mail
email = emails[0]
print("From:", email.sender)
print("To:", email.to)
print("Subject:", email.subject)
display(HTML(email.content))

## Versenden der E-Mails

In diesem Abschnitt werden die erstellten E-Mails als Entwürfe im eigenen Postfach gespeichert, damit sie überprüft und anschließend manuell versendet werden können. Alternativ gibt es die Möglichkeit, die E-Mails direkt zu versenden.

In [ ]:
# The routine creates the e-mails as drafts, but does not send them - this mustmbe done manually by the user.
# If you want to send the e-mails directly, set the SEND_DIRECTLY variable to True.
# Important: This will send the e-mails to the recipients immediately! Only use this option if you are
# absolutely sure that the e-mails are correct and ready to be sent.
SEND_DIRECTLY = False

In [ ]:
import time
from tqdm import tqdm
from src.email import EmailClient

# Initialize the email client
email_client = EmailClient(**client_config.__dict__)

# Create the drafts
num_emails = 0
for email in tqdm(
    emails[:3],
    desc=f"{'Sending' if SEND_DIRECTLY else 'Drafting'} E-Mails",
):
    email_client.draft(email)
    num_emails += 1
    time.sleep(1)

print(f'Processed {num_emails} e-mails.')